Determine the transformation of an aperture diameter of 7.5 kpc at the redshift of abell 370 to pixels. First transform the diameter in absolute size (kpc) to apparent size (arcseconds). Then transform apparent size to pixels using WCS.

In [2]:
import numpy as np 

from astropy.io import fits
from astropy import units as u 
from astropy import cosmology
from astropy.wcs import WCS

### Load Data

In [73]:
input_data_path = "https://archive.stsci.edu/pub/hlsp/frontier/abell2744/images/hst/v1.0-epoch1/hlsp_frontier_hst_wfc3-60mas_abell2744_f105w_v1.0_drz.fits"

hdul = fits.open(input_data_path)
hdu = hdul[0]

data = hdu.data
header = hdu.header
wcs = WCS(header)

### Given

In [74]:
# Given 
aperture_diameter = (7.5 * u.kpc).to(u.Mpc)

NED data for abell 2744:

http://ned.ipac.caltech.edu/byname?objname=Abell%202744&hconst=70.0&omegam=0.279&omegav=0.721&wmap=3&corr_z=1

In [75]:
"""
Cosmology-Corrected Quantities [H0 = 70.0 km/sec/Mpc, Ωmatter = 0.279, Ωvacuum = 0.721]
[Redshift 0.307055 as corrected to the Reference Frame defined by the 3K CMB]
Type	Values
Luminosity Distance	1.6e+03 Mpc (m-M) = 41.02 mag
Angular-Size Distance	938 (m-M) = 39.86 mag
Co-Moving Radial Distance	1.23e+03 (m-M) = 40.44 mag
Co-Moving Tangential Distance	1.23e+03 (m-M) = 40.44 mag
Co-Moving Volume	7.72 Gpc^3
Light Travel-Time	3.499 Gyr
Age at Redshift 0.307055	10.245 Gyr
Age of Universe	13.744 Gyr
Scale (Cosmology Corrected)	4548 pc/arcsec = 4.548 kpc/arcsec = 272.85 kpc/arcmin = 16.37 Mpc/degree
Surface Brightness Dimming	Flux Density per Unit Area = 0.34263; Magnitude per Unit Area = 1.163 mag
""";

In [76]:
abell2744_z =  0.30800
abell2744_distance = 1230 * u.Mpc # Comoving

### Equations

\begin{align}
\theta = \frac{(z+1) * Diameter}{Distance_{Comoving}} =  \frac{Diameter}{Distance_{Angular Diameter} } \\
\end{align}

### Compute angular diameter distance

In [77]:
def compute_angular_diameter_distance(d, z):
    return d/(z+1)

In [78]:
angular_diameter_distance = compute_angular_diameter_distance(abell2744_distance, abell2744_z)
angular_diameter_distance

<Quantity 940.36697248 Mpc>

In [79]:
# check astropy cosmology
cosmology.WMAP5.angular_diameter_distance(abell2744_z)

<Quantity 937.71584752 Mpc>

### Find angular diameter

In [86]:
angular_diameter = np.tan((aperture_diameter / angular_diameter_distance).value) * u.rad 
angular_diameter 

<Quantity 7.97560976e-06 rad>

In [87]:
# check astropy cosmology
(cosmology.WMAP5.arcsec_per_kpc_proper(abell2744_z) * aperture_diameter.to('kpc')).to('rad')

<Quantity 7.99815853e-06 rad>

### Convert to arcsec

In [96]:
angular_diameter = 0.7*u.arcsec#angular_diameter.to("arcsec")
angular_diameter

<Quantity 0.7 arcsec>

### Find Pixel Size 

In this section we convert the angular size into pixel size. The way we do this is by adding the angular size to the `CRVAL` of the WCS and then converting that world coordinate to pixel values. After getting the new pixel values we subtract the `CRPIX` values to find the difference (angular size). Because RA-DEC is an equatorial coordinate system, I have decided its best to add the angular size to the Dec component of the WCS CRVAL (Dec axis is a great circle). It looks like the image `CRVAL` is sufficiently away from the pols of the celestial sphere, so no need to worry about loops around +/- 90 degrees.  

In [97]:
# Load center pixel from data's WCS 

ra_0 = wcs.wcs.crval[0] * u.Unit(wcs.wcs.cunit[0])
dec_0 = wcs.wcs.crval[1] * u.Unit(wcs.wcs.cunit[1])

print(ra_0, ",",  dec_0)

3.5832679 deg , -30.3880082 deg


In [98]:
# Add angular_diameter to center Dec value 

ra_1  = ra_0
dec_1 = dec_0 + angular_diameter

print(ra_1, ",",  dec_1)

3.5832679 deg , -30.387813755555555 deg


In [99]:
# Convert ra_1, dec_1 into pixels

world = np.array([[ra_1.value, dec_1.value],], dtype=np.float64) * u.deg
pixcrd = wcs.wcs_world2pix(world, 1)

pixcrd

array([[2675.00000008, 3086.66666198]])

In [100]:
# Look at the center pixel values
wcs.wcs.crpix

array([2675., 3075.])

In [101]:
# Find the difference b/w the center pixel and the angular offset pixel:
pixel_diff =  pixcrd - wcs.wcs.crpix 
assert abs(pixel_diff[0,0]) < 1e-6
pixel_diff[0,1]

11.666661982622372

In [102]:
# Find ceiling of pixel diff to conclude pixel size
pixel_size = int(np.ceil(pixel_diff[0,1]))

pixel_size

12

In [103]:
# radius 
pixel_size // 2

6